In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pandas import DataFrame, Series
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing
import warnings
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
import xgboost
from sklearn.metrics import *
from imblearn.under_sampling import *
from imblearn.over_sampling import *
from imblearn.combine import *
from sklearn.model_selection import train_test_split


warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 30000)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

/home/dockeruser/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# 모델링

In [2]:
df_pre = pd.read_csv('data/df_pre.csv')
print(np.shape(df_pre))
df_pre.tail()

(26729, 24)


,Unnamed: 0,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Color_pre,DayuponOutcome,AboutAge,BreedMix,Breed_pre,Year,Month,YearMonth,Hour,Season,Quarter,Group,Size
26724,26724,18631,0,14983,4,Partner,0,1,2,640,111,2,30,2,1,80,2,4,19,11,1,1,8,5
26725,26725,26055,0,22209,4,SCRP,0,3,24,640,104,2,90,2,1,80,3,0,27,18,3,0,8,5
26726,26726,16731,1,13463,0,NaN,1,2,30,1022,353,22,1460,20,1,147,2,2,17,13,1,0,12,9
26727,26727,7450,0,5074,4,Partner,0,1,29,640,5,0,28,2,1,80,1,3,6,12,1,1,8,5
26728,26728,20751,0,16611,4,SCRP,0,1,5,640,111,2,365,2,1,80,2,6,21,9,2,2,8,5


In [3]:
del df_pre['Unnamed: 0']

In [4]:
print(np.shape(df_pre))
df_pre.tail()

(26729, 23)


,AnimalID,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color,Color_pre,DayuponOutcome,AboutAge,BreedMix,Breed_pre,Year,Month,YearMonth,Hour,Season,Quarter,Group,Size
26724,18631,0,14983,4,Partner,0,1,2,640,111,2,30,2,1,80,2,4,19,11,1,1,8,5
26725,26055,0,22209,4,SCRP,0,3,24,640,104,2,90,2,1,80,3,0,27,18,3,0,8,5
26726,16731,1,13463,0,NaN,1,2,30,1022,353,22,1460,20,1,147,2,2,17,13,1,0,12,9
26727,7450,0,5074,4,Partner,0,1,29,640,5,0,28,2,1,80,1,3,6,12,1,1,8,5
26728,20751,0,16611,4,SCRP,0,1,5,640,111,2,365,2,1,80,2,6,21,9,2,2,8,5


# AdaBoost 

In [11]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
columns = ['Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'DayuponOutcome', 'Breed', 'Color', 'BreedMix','Hour']
df_x = df_pre[columns]
df_y = df_pre.iloc[:, [3]]

model_ada = AdaBoostClassifier(DecisionTreeClassifier(random_state=0), 
                               algorithm="SAMME", n_estimators=300)
model_ada.fit(df_x, df_y)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle=True)
print('log loss: {}'.format(np.mean(-cross_val_score(model_ada, df_x, df_y, scoring="neg_log_loss", cv=cv))))
print()

log loss: 1.514926001826654



# Random Forest 

In [14]:
columns = ['Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'DayuponOutcome', 'Breed', 'Color', 'BreedMix','Hour']

df_x = df_pre[columns]
df_y = df_pre.iloc[:, [3]]

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import *

rf = RandomForestClassifier(n_estimators=100, random_state=0)
rf.fit(df_x, df_y)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle=True)
print('log loss: {}'.format(np.mean(-cross_val_score(rf, df_x, df_y, scoring="neg_log_loss", cv=cv))))
print()

log loss: 1.0271558930276539



# Logistic Regression

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import *

columns = ['Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'DayuponOutcome', 'Breed', 'Color', 'BreedMix','Hour']

df_x = df_pre[columns]
df_y = df_pre.iloc[:, [3]]

LR = LogisticRegression().fit(df_x, df_y)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle=True)
print('log loss: {}'.format(np.mean(-cross_val_score(LR, df_x, df_y, scoring="neg_log_loss", cv=cv))))
print()

log loss: 1.0430491317461645



# Gradient Boosting 

In [5]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

columns = ['Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'DayuponOutcome', 'Breed', 'Color', 'BreedMix','Hour']
df_x = df_pre[columns]
df_y = df_pre.iloc[:, [3]]

model_grad = GradientBoostingClassifier(n_estimators=100, random_state=0).fit(df_x, df_y)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle=True)
print('log loss: {}'.format(np.mean(-cross_val_score(model_grad, df_x, df_y, scoring="neg_log_loss", cv=cv))))
print()

log loss: 0.7862887775735734



In [7]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import *
from sklearn.model_selection import train_test_split

columns = ['Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'DayuponOutcome', 'Breed', 'Color', 'BreedMix','Hour']
df_x = df_pre[columns]
df_y = df_pre.iloc[:, [3]]

model_grad = GradientBoostingClassifier(n_estimators=200, max_depth=6, learning_rate=0.05, random_state=0).fit(df_x, df_y)

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle=True)
print('log loss: {}'.format(np.mean(-cross_val_score(model_grad, df_x, df_y, scoring="neg_log_loss", cv=cv))))
print()

log loss: 0.7581365952626343



# XGBoost 

In [9]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
cv = KFold(20, shuffle=True)

columns = ['Name', 'DateTime', 'AnimalType', 'SexuponOutcome', 'DayuponOutcome', 'Breed', 'Color', 'BreedMix','Hour']

df_x = df_pre[columns]
df_y = df_pre.iloc[:, [3]]

model_xgb = xgboost.XGBClassifier(n_estimators=100)
model_xgb.fit(df_x, df_y)

print('log loss: {}'.format(np.mean(-cross_val_score(model_xgb, df_x, df_y, scoring="neg_log_loss", cv=cv))))
print()

log loss: 0.7944662875695214

